# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.12.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
pip install tensorflow

In [ ]:
pip install pandas

In [ ]:
pip install pyspark

In [ ]:
pip install pandas scikit-learn pyarrow

In [6]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

# 2. Collect Keypoint Values for Training and Testing

In [7]:

df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks")

In [ ]:
df_from_parquet.head(0)

In [9]:
df=df_from_parquet.groupby(["Label","Sample_Number"])["Frame_Number"].count()

In [ ]:
display(df)

## 2.1 Label indexing

In [11]:

df_label=df_from_parquet["Label"].unique()
label_map = {label:num for num, label in enumerate(df_label)}
df_label_index = pd.DataFrame(list(label_map.items()), columns=['Label', 'Label_id'])

In [ ]:
df_label_index

## 2.2 sample_type indexing

In [13]:
df_sample_type=df_from_parquet["sample_type"].unique()
sample_type_map = {sample_type:num for num, sample_type in enumerate(df_sample_type)}
df_sample_type_index = pd.DataFrame(list(sample_type_map.items()), columns=['sample_type', 'sample_type_id'])

In [ ]:
df_sample_type_index

## 2.3 Data Frame Clean up

In [15]:
inner_join = df_from_parquet.merge(df_label_index, on='Label', how='left')
df_raw=inner_join.merge(df_sample_type_index, on='sample_type', how='left')


In [16]:
df_clean=df_raw[["Label_id","sample_type_id","Sample_Number","Frame_Number","Keypoints"]]


In [59]:
label_info = [{'Label': label, 'Label_id': label_id} for label, label_id in label_id_mapping.items()]


In [62]:
data = json.loads(json_output)
print(data[0])

{'Label': 'He is going into the room', 'Label_id': 18, 'sample_type': 'Video', 'Sample_Number': 0, 'frames_keypoints': [{'Frame_Number': 1.0, 'Keypoints': [0.49935734272003174, 0.36126840114593506, -0.6683312654495239, 0.9997991919517517, 0.5147819519042969, 0.3265095353126526, -0.6404985189437866, 0.9996405839920044, 0.5223203301429749, 0.3272407054901123, -0.6406830549240112, 0.9995532631874084, 0.5293248295783997, 0.32920074462890625, -0.6407297849655151, 0.9995288848876953, 0.48863551020622253, 0.32562536001205444, -0.6432749629020691, 0.9997292160987854, 0.4806400239467621, 0.32629841566085815, -0.6433780789375305, 0.9997369647026062, 0.4729194939136505, 0.3278321623802185, -0.6437049508094788, 0.9997885823249817, 0.5409995317459106, 0.34637266397476196, -0.4139142334461212, 0.9994138479232788, 0.4645395874977112, 0.34331125020980835, -0.4210797846317291, 0.9998000264167786, 0.5154014825820923, 0.40166616439819336, -0.5797024369239807, 0.9999419450759888, 0.48525896668434143, 0.40

In [57]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


# 3. Train Test Split

In [18]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [19]:
X=df_clean[["Label_id","sample_type_id","Sample_Number"]].drop_duplicates()

In [ ]:
X

In [21]:
y=df_raw[["Label_id","Sample_Number","Frame_Number","Keypoints"]]

In [ ]:
y.head()

In [23]:

y_new=y.groupby(["Label_id","Sample_Number"])["Keypoints"].apply(list).reset_index(name='Holistic_landmarks')

In [24]:
Y=y_new["Holistic_landmarks"]

In [ ]:
print(y)

In [26]:
max_length=y.apply(len).max()

In [27]:
new_column_names = [f'{"col"}_{i}' for i in range(1, max_length + 1)]

In [ ]:
display(y)

In [ ]:
y[486]

In [29]:
y.to_csv('output_file.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

# Assuming each array is in the file as a single line of text
#with open("your_file.txt", "r") as file:
#    data = file.readlines()

# Process each line, convert it to an array, and store it in a list
arrays = [np.fromstring(line.strip().strip("[]"), sep=",") for line in y]


# Create dataframe from the list of arrays
df = pd.DataFrame(arrays)
print(df)


In [ ]:
print(df.col_1)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y.shape

# 4. Build and Train LSTM Neural Network

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [35]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(label.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()